<a href="https://colab.research.google.com/github/anaswaraku/Kidney-disease-detection/blob/main/VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
from imageio.v2 import imread
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data_folders = ["Cyst", "Stone", "Tumor", "Normal"]
flat_data_arr = []
target_arr = []
base_dir = '/content/drive/MyDrive/Kidney-disease-detetction/Trial/TrialTrain'
output_dir = '/content/drive/MyDrive/Kidney-disease-detetction/Trial/TrialTrain/Preprocessed'

In [ ]:
# Function to resize images to (224, 224) pixels
def resize_images(image_dir):
        image = cv2.imread(image_dir, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(image, (224, 224))
        return img

In [ ]:
# Function for identical blurring
def Gaussian_blur (img):
    kernel_size = (5, 5)  # Adjust kernel size as needed
    sigma = 0  # Sigma is not directly specified in OpenCV's GaussianBlur
    filtered_image = cv2.GaussianBlur(img, kernel_size, sigma)
    return filtered_image

In [ ]:
def clahe(image):
    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
    clahe_image=clahe.apply(image)
    return clahe_image

In [ ]:
def histogram_equalization(img):
  img_yuv = cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
  img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
  equalized_img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
  return(equalized_img)

In [ ]:
def min_max_scaling(img):
  scaler = MinMaxScaler()
  img_scaled = scaler.fit_transform(img.flatten().reshape(-1, 1)).reshape(img.shape)
  return(img_scaled)

In [ ]:
# Function for image augmentation
def image_augmentation(img):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    # Remove the batch size dimension
    aug_img = img[0]
    return aug_img


In [ ]:

for folder in data_folders:
    folder_path = os.path.join(base_dir, folder)
    print("Folder Path:", folder_path)
    output_folder = os.path.join(output_dir, folder)
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            # Preprocess image
            preprocessed_image = resize_images(image_path)
            preprocessed_image = Gaussian_blur(preprocessed_image)
            preprocessed_image = clahe(preprocessed_image)
            preprocessed_image = min_max_scaling(preprocessed_image)
            preprocessed_image = image_augmentation(preprocessed_image)

            # Save preprocessed image
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, preprocessed_image)
print('Preprocess Completed')

Folder Path: /content/drive/MyDrive/Kidney-disease-detetction/Trial/TrialTrain/Cyst
Folder Path: /content/drive/MyDrive/Kidney-disease-detetction/Trial/TrialTrain/Stone
Folder Path: /content/drive/MyDrive/Kidney-disease-detetction/Trial/TrialTrain/Tumor
Folder Path: /content/drive/MyDrive/Kidney-disease-detetction/Trial/TrialTrain/Normal
Preprocess Completed


**VGG-16**

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
data_dir = '/content/drive/MyDrive/Kidney-disease-detetction/Trial/TrialTrain/Preprocessed'
classes = ['Cyst', 'Normal', 'Tumor', 'Stone']

# Create directories for train, test, and validation data
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
val_dir = os.path.join(data_dir, 'validation')
for directory in [train_dir, test_dir, val_dir]:
    for class_name in classes:
        os.makedirs(os.path.join(directory, class_name), exist_ok=True)

# Split dataset into train, test, and validation sets
for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    images = [f for f in os.listdir(class_dir) if f.endswith('.jpg')]
    train_images, test_val_images = train_test_split(images, test_size=0.3, random_state=42)
    test_images, val_images = train_test_split(test_val_images, test_size=0.5, random_state=42)

    # Move images to corresponding directories
    for image in train_images:
        src = os.path.join(class_dir, image)
        dest = os.path.join(train_dir, class_name, image)
        shutil.copy(src, dest)
    for image in test_images:
        src = os.path.join(class_dir, image)
        dest = os.path.join(test_dir, class_name, image)
        shutil.copy(src, dest)
    for image in val_images:
        src = os.path.join(class_dir, image)
        dest = os.path.join(val_dir, class_name, image)
        shutil.copy(src, dest)

print("Dataset split into train, test, and validation sets.")


Dataset split into train, test, and validation sets.


In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [30]:

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Define paths to training and validation data
train_data_dir = '/content/drive/MyDrive/Preprocessed/train'
validation_data_dir = '/content/drive/MyDrive/Preprocessed/validation'

In [31]:
# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)
test_data_dir = '/content/drive/MyDrive/Preprocessed/test'
test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'

)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 2099 images belonging to 4 classes.
Found 9791 images belonging to 4 classes.
Found 2100 images belonging to 4 classes.


In [27]:
# Build the VGG-16 model
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(img_height, img_width, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))  # 5 classes: cyst, normal, tumor, stone, and kidney

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])


In [ ]:

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=20)

# Save the trained model
model.save('vgg16_kidney_classification.h5')


Epoch 1/20
  3/305 [..............................] - ETA: 6:25:59 - loss: 1.4173 - accuracy: 0.2292

In [ ]:
from keras.models import load_model

# Load the trained model
trained_model = load_model('vgg16_kidney_classification.h5')

# Evaluate the model on the test set
test_loss, test_accuracy = trained_model.evaluate(test_generator)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)


63/63 [==============================] - 1293s 20s/step - loss: 1.3871 - accuracy: 0.2522
Test Loss: 1.3870989084243774
Test Accuracy: 0.2522343695163727


In [ ]:
print('Test Loss:', test_loss*100)
print('Test Accuracy:', test_accuracy*100)

Test Loss: 138.72514963150024
Test Accuracy: 25.0
